In [1]:
import tensorflow as tf
Model        = tf.keras.models.Model
Input        = tf.keras.layers.Input
Dense        = tf.keras.layers.Dense
Conv2D       = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D
UpSampling2D = tf.keras.layers.UpSampling2D
#mnist        = tf.keras.datasets.mnist
save_model   = tf.keras.models.save_model
load_model   = tf.keras.models.load_model

In [3]:
bottleneck_dim = 64

# Architecture
layer_input   = Input(shape=(784,))
layer_encode1 = Dense(256, activation='relu')(layer_input)
layer_encode2 = Dense(128, activation='relu')(layer_encode1)

layer_bottleneck = Dense(bottleneck_dim, activation='relu')(layer_encode2)

layer_decode1 = Dense(128, activation='relu')(layer_bottleneck)
layer_decode2 = Dense(256, activation='relu')(layer_decode1)
layer_output  = Dense(784, activation='sigmoid')(layer_decode2)

# Autoencoder Model
autoencoder = Model(layer_input, layer_output)

# Encoder Model
encoder = Model(layer_input, layer_bottleneck)

# Decoder Architecture
Dlayer_input   = Input(shape=(bottleneck_dim,))
Dlayer_decode1 = autoencoder.layers[-3](Dlayer_input)
Dlayer_decode2 = autoencoder.layers[-2](Dlayer_decode1)
Dlayer_output  = autoencoder.layers[-1](Dlayer_decode2) # extract last layer

# Decoder Model
decoder = Model(Dlayer_input, Dlayer_output)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [6]:
def make_square(img):
    cols,rows = img.size
    
    if rows>cols:
        pad = (rows-cols)/2
        img = img.crop((pad,0,cols,cols))
    else:
        pad = (cols-rows)/2
        img = img.crop((0,pad,rows,rows))
    
    return img

In [9]:
import os
from PIL import Image, ImageFile
import requests
#import cv2
import numpy as np
from io import BytesIO
import matplotlib.pyplot as plt
imshow = plt.imshow
from IPython.display import display, HTML

path = r"D:\Public\fyp-code\tensorflow2-fyp\tensorflow2-fyp-compress\PetImages\Cat"

#for img in os.listdir(path): # iterate over each image per dogs and cats
#    img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) # convert to array # try remove cv2.IMREAD_GRAYSCALE
#    plt.imshow(img_array, cmap="gray") # graph it
#    plt.show() # display
#    break # we just want one for now so break

# Checking resized image
#IMG_SIZE = 28 # IMPORTANT: must match input size = IMG_SIZE^2

#new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
#plt.imshow(new_array, cmap="gray")
#plt.show() # TODO fix array resize

training_data = []

IMAGE_WIDTH = 200
IMAGE_HEIGHT = 200
IMAGE_CHANNELS = 3

for img in os.listdir(path):
    ImageFile.LOAD_TRUNCATED_IMAGES = False
    print(img)
    #response = requests.get(url)
    img = Image.open(BytesIO(os.path.join(path,img))
    img.load()
    img = make_square(img)
    img = img.resize((IMAGE_WIDTH,IMAGE_HEIGHT),Image.ANTIALIAS)
    training_data.append(np.asarray(img))


SyntaxError: invalid syntax (<ipython-input-9-224fbc0973ca>, line 37)

In [6]:
training_data = []

def create_training_data():
    #path = os.path.join(DATADIR, category) # path to cats or dogs dir
    #class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) # try remove cv2.IMREAD_GRAYSCALE
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            training_data.append([new_array])
            #training_data.append([new_array, class_num]) # todo use this to store original size
        except Exception as e:
            print('Skipping problematic image: ' + img)
            pass

create_training_data()

Skipping problematic image: 10125.jpg
Skipping problematic image: 10501.jpg
Skipping problematic image: 10820.jpg
Skipping problematic image: 11210.jpg
Skipping problematic image: 11565.jpg
Skipping problematic image: 11874.jpg
Skipping problematic image: 11935.jpg
Skipping problematic image: 140.jpg
Skipping problematic image: 2663.jpg
Skipping problematic image: 3300.jpg
Skipping problematic image: 3491.jpg
Skipping problematic image: 4833.jpg
Skipping problematic image: 5553.jpg
Skipping problematic image: 660.jpg
Skipping problematic image: 7968.jpg
Skipping problematic image: 7978.jpg
Skipping problematic image: 8470.jpg
Skipping problematic image: 850.jpg
Skipping problematic image: 9171.jpg
Skipping problematic image: 936.jpg
Skipping problematic image: 9565.jpg
Skipping problematic image: 9778.jpg


In [7]:
print(len(training_data))

12476


In [ ]:
%%time
# Fit data
autoencoder.fit(training_data, training_data,
                epochs=50, batch_size=256, shuffle=True,
                validation_data=(training_data, training_data)) # WARINING: RAM will rise

In [ ]:
encoded_imgs = encoder.predict(training_data)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(decoded_imgs[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()

In [ ]:
import numpy 
import math
import cv2

def psnr(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

n = 10
for i in range(n):
    original = x_test[i]
    contrast = decoded_imgs[i]
    d=psnr(original,contrast)
    print("%.1f" % d)

In [ ]:
autoencoder.summary()

In [ ]:
encoder.summary()

In [ ]:
decoder.summary()

In [ ]:
save_model(autoencoder, "model/autoencoderv4-784-256-128-64-128-256-784.h5")
save_model(encoder, "model/encoderv4-784-256-128-64.h5")
save_model(decoder, "model/decoderv4-64-128-256-784.h5")